## Vorbereitung LM Studio

Zunächst müssen wir LM Studio installieren und mindestens ein LLM herunterladen. Als Beispiel hier habe ich das LLAMA 3.1 Model (8B) geladen.

Dieses Modell müssen wir in LM Studio lokal starten. Es läuft dann lokal auf dem Rechner, und wird automatisch gefunden.

Dafür (und sowieso) brauchen wir die richtige Library und müssen das Modell instantiieren.

In [5]:

import lmstudio as lms

model = lms.llm("meta-llama-3.1-8b-instruct")

### Wir probieren aus ob das prinzipiell klappt 

In [6]:
result = model.respond("Who are you?")
print(result)

I'm an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."


## Laden der Reden

Wir wollen ja, das LLM benutzen, um die Reden zu charakterisieren. Also müssen wir die Reden laden. Das machen wir wie immer.

In [7]:
import jsonlines

legislatur = 20

alleReden = []
with jsonlines.open(f'../../data/speeches_{legislatur}.jsonl') as f:
    for line in f.iter():
        #for line in list(f):
        alleReden.append(line)

alleReden.sort(key = lambda x:x['date'])
print(len(alleReden))

25643


In [8]:
welche = 102
eineRede = alleReden[welche]
print(eineRede['text'])

Von mir aus nicht, Herr Spahn. Bleiben Sie ruhig sitzen. Herr Spahn, ich finde das eigentlich gut, was Sie gesagt haben, weil wir jetzt in ein Gespräch kommen, und das biete ich der Union, das bieten wir alle der Union an, auch über diese Abstimmung heute hinaus. Wir sind schon im Hauptausschuss auf die Union zugegangen. Ich habe es in vier Jahren Opposition nicht einmal erlebt – nicht einmal! –, dass wir noch während einer laufenden Sitzung eine Änderung an einem Vorschlag der Regierungsfraktionen vorgenommen haben. Das ist diese Woche passiert. Ich weiß nicht, wie weit wir noch auf Sie zugehen sollen. Aber woran es krankt, ist: Sie machen keine konkreten Vorschläge. Sie sagen uns nicht, was genau Sie wollen. Kommen Sie auf uns zu, und wir reden gerne weiter. Vielen Dank. – Als nächster Redner hat jetzt das Wort für die FDP-Fraktion Michael Theurer. 


## Eine erste Anfrage

In [9]:
system_msg = 'Du bist eine Experte in deutscher Politik. Dein Ziel ist es, den Menschen in Deutschland aktuelle politische Diskussionen verständlicher zu machen.'

user_msg = f"""Kannst mir den Inhalt folgender Bundestagsrede zusammenfassen? Hier ist die Rede: '{eineRede['text']}' """

chat = lms.Chat()
chat.add_system_prompt(system_msg)
chat.add_user_message(user_msg)
result = model.respond(chat)

print(result)

Die Rede wurde von Olaf Scholz gehalten, einem Mitglied der SPD-Fraktion im Deutschen Bundestag. Hier ist eine Zusammenfassung des Inhalts:

Olaf Scholz beginnt seine Rede mit einer Anmerkung zu einem Vorschlag von Karl Lauterbach (SPD) und kritisiert damit die Union, dass diese nicht konstruktiv an dem Gespräch teilnimmt. Er bezeichnete den Vorschlag als eine gute Gelegenheit für ein Gespräch mit der Union.

Er lobt die Geduld der SPD-Fraktion, die in vier Jahren Opposition noch kein Mal einen Änderungsvorschlag während einer laufenden Sitzung der Regierungsfraktionen vorgenommen hat. Er fragt sich, wie weit sie auf die Union zugehen sollen.

Olaf Scholz kritisiert jedoch auch die Union, dass diese keine konkreten Vorschläge macht und nicht sagt, was genau sie wollen. Er bittet die Union, sich auf das Gespräch einzulassen und gemeinsam weiter zu diskutieren.

Die Rede endet mit einer freundlichen Note, da Olaf Scholz Michael Theurer (FDP) als nächsten Redner einlädt.


## Let's play!

In [11]:
welche = 1001
eineRede = alleReden[welche]

system_msg = 'Du bist eine Experte in deutscher Politik. Dein Ziel ist es, den Menschen in Deutschland aktuelle politische Diskussionen verständlicher zu machen.'

user_msg = f"""Kannst mir den Inhalt folgender Bundestagsrede zusammenfassen? Hier ist die Rede: '{eineRede['text']}' """

chat = lms.Chat()
chat.add_system_prompt(system_msg)
chat.add_user_message(user_msg)
result = model.respond(chat)

print(result)


Die Rede ist von einem Bundestagsabgeordneten der Partei Bündnis 90/Die Grünen, wahrscheinlich Robin Wagener. Er kritisiert in seiner Rede die Politik der Union (CDU/CSU) in Bezug auf das Verhältnis zu Russland.

Der Redner beginnt mit einer Anspielung auf ein russisches Lied und fragt, ob die Russen Krieg wollen. Er antwortet selbst, dass die Antwort "nein" ist und dass auch Deutschland keinen Krieg in Europa will.

Der Redner wirft der Union vor, dass ihre Politik gegenüber Russland von Propaganda geprägt ist und dass sie Russland alleinige Verantwortung für die aktuellen Spannungen gibt. Er behauptet jedoch, dass es den Westen war, der über Jahre hinweg russische Sicherheitsinteressen ignoriert hat, was zu einer gereizten Reaktion von Russland geführt habe.

Der Redner kritisiert auch, dass die Union Russland vorgeworfen wird, die Grenzen anderer Staaten zu missachten, während sie selbst dabei ist, NATO-Truppen in der Nähe russischer Grenzen zu stationieren. Er erinnert an den Angri